In [ ]:
# !/home/ec2-user/anaconda3/envs/Braket/bin/python -m pip install tensorflow

In [ ]:
# !/home/ec2-user/anaconda3/envs/Braket/bin/python -m pip install scikit-learn

In [ ]:
# !/home/ec2-user/anaconda3/envs/Braket/bin/python -m pip install seaborn

In [ ]:
# pip install torch torchvision

In [ ]:
from keras.models import load_model

encoder_model = load_model("latest_latest.h5")
encoder_model.summary()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv('aug_weathered_5plastics_baselineCorr.csv')

In [ ]:
data

In [ ]:
data['plastic'].value_counts()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))  # Normalize between 0 and 1

remove_classes = ["PS", "PC", "NC", "/", "ABS"]
# Filter out rows where 'Plastic' is in remove_classes
data = data[~data['plastic'].isin(remove_classes)]

print(data.shape)

wave_numbers = data.columns[2:]
spectra = data.iloc[:, 2:]

In [ ]:
wave_numbers

In [ ]:
spectra

In [ ]:
data

In [ ]:
all_data = []
for col in range (len(spectra)):
    y = spectra.iloc[col].values.astype(float)
    # Read the file assuming space-separated values
    temp = y.reshape(-1, 1)

    # Normalize between 0 and 1
    temp = scaler.fit_transform(temp)

    # Flatten to 1D and store
    temp = temp.reshape(1, -1)

      # Append to the list
    all_data.append(temp)

In [ ]:
all_data

In [ ]:
y.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
labels = data['plastic'].astype(str)

In [ ]:
labels

In [ ]:
X =  np.array(all_data)
y = encoder.fit_transform(labels)

In [ ]:
X.shape

In [ ]:
X

In [ ]:
y

In [ ]:
print(encoder.classes_)

In [ ]:
X.shape

# 1. AE

In [ ]:
AE_op = encoder_model.predict(X)

In [ ]:
AE_op.shape

# 2. PCA

In [ ]:
from sklearn.decomposition import PCA
X_pca = X.squeeze(axis=1)

In [ ]:
pca = PCA(n_components=16)

In [ ]:
X_pca = pca.fit_transform(X_pca)
X_pca.shape

# 3. LDA

In [ ]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Assuming your data X is a NumPy array of shape (1200, 1, 16)
X_AE = np.squeeze(AE_op, axis=1)  # Now X is of shape (1200, 16)

In [ ]:
# Initialize LDA with desired number of components; remember that maximum is (n_classes - 1)
lda = LinearDiscriminantAnalysis(n_components=3)
# Fit LDA and transform the data
lda_ae = lda.fit_transform(X_AE, y)
print("Shape of AR data after LDA transformation:", lda_ae.shape)

In [ ]:
lda2 = LinearDiscriminantAnalysis(n_components=3)
# Fit LDA and transform the data
lda_pca = lda2.fit_transform(X_pca, y)
print("Shape  of PCA data after LDA transformation:", lda_pca.shape)

# 4. VQC

In [ ]:
vqc_data = np.concatenate((lda_ae, lda_pca), axis=1)
print("Concatenated vertically:\n", vqc_data.shape)

In [ ]:
# !pip install pennylane

In [ ]:
# import pennylane as qml
# print(qml.__version__)

In [ ]:
# !pip uninstall torch torchvision torchaudio -y
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [ ]:
# import torch
# print(torch.__version__)
# print(torch.cuda.is_available())

In [ ]:
import torch
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
vqc_data = scaler.fit_transform(vqc_data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vqc_data, y, test_size=0.2, random_state=42,stratify=y)

X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)

In [ ]:
# !nvcc --version
# !nvidia-smi

In [ ]:
# !sudo yum update -y

In [ ]:
# !sudo ubuntu-drivers devices

In [ ]:
# import pennylane as qml
# import numpy as np
# import torch
# from torch import nn
# from torch.optim import Adam
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# # Hyperparameters
# num_qubits = 6       # 6 qubits
# num_layers = 1       # Reduced depth for speed
# num_classes = 5      # 5-class classification
# epochs = 500        # Fewer epochs for faster training
# batch_size = 16      # Mini-batch size
# learning_rate = 0.01 # Learning rate

# # Define Quantum Device (6 qubits)
# #dev = qml.device("default.qubit", wires=num_qubits)
# from braket.tracking import Tracker
# t = Tracker().start()
# device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
# dev = qml.device('braket.aws.qubit', device_arn=device_arn, wires=n_w)
# # Quantum Circuit with Optimized Speed
# @qml.qnode(dev, interface="torch")
# def quantum_circuit(inputs, weights):
#     # Faster Encoding: AngleEmbedding (Y rotations)
#     qml.AngleEmbedding(inputs, wires=range(num_qubits), rotation="Y")

#     # Variational Quantum Circuit (VQC)
#     for w in weights:
#         for i in range(num_qubits):
#             qml.RY(w[i], wires=i)
#             qml.RZ(w[i], wires=i)  # Removed RX for speed

#         # Star Entanglement: Faster than All-to-All
#         for i in range(1, num_qubits):
#             qml.CNOT(wires=[0, i])  # Connect all to qubit 0

#     return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]  # Expectation values

# # Define Torch Model with Quantum Layer
# class QuantumClassifier(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.q_weights = nn.Parameter(0.01 * torch.randn(num_layers, num_qubits))  # Trainable weights
#         self.fc = nn.Linear(num_qubits, num_classes)  # Fully connected layer

#     def forward(self, x):
#         q_out = torch.tensor([quantum_circuit(xi, self.q_weights) for xi in x], dtype=torch.float32)
#         return self.fc(q_out)  # Map quantum features to class probabilities

# # Initialize Model, Loss, Optimizer
# model = QuantumClassifier()
# loss_fn = nn.CrossEntropyLoss()
# optimizer = Adam(model.parameters(), lr=learning_rate)

In [ ]:
import pennylane as qml
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Start tracking AWS Braket task costs
from braket.tracking import Tracker
t = Tracker().start()
from braket.aws import AwsDevice

# Hyperparameters
num_qubits = 6       # 6 qubits
num_layers = 1       # Reduced depth for speed
num_classes = 5      # 5-class classification
epochs = 30        # Fewer epochs for faster training
batch_size = 16      # Mini-batch size
learning_rate = 0.01 # Learning rate

# Define AWS Braket Quantum Device (SV1 simulator)
device_arn = "arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1"
device = AwsDevice(device_arn)
dev = qml.device('braket.aws.qubit', device_arn=device_arn, wires=num_qubits, shots=20)

# Quantum Circuit with Optimized Speed
@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs, weights):
    # Faster Encoding: AngleEmbedding (Y rotations)
    qml.AngleEmbedding(inputs, wires=range(num_qubits), rotation="Y")

    # Variational Quantum Circuit (VQC)
    for w in weights:
        for i in range(num_qubits):
            qml.RY(w[i], wires=i)
            qml.RZ(w[i], wires=i)  # Removed RX for speed

        # Star Entanglement: Faster than All-to-All
        for i in range(1, num_qubits):
            qml.CNOT(wires=[0, i])  # Connect all to qubit 0

    return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]  # Expectation values

# Define Torch Model with Quantum Layer
class QuantumClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.q_weights = nn.Parameter(0.01 * torch.randn(num_layers, num_qubits))  # Trainable weights
        self.fc = nn.Linear(num_qubits, num_classes)  # Fully connected layer

    def forward(self, x):
        # Compute quantum circuit output for each input in batch
        q_out = torch.stack([torch.tensor(quantum_circuit(xi, self.q_weights), dtype=torch.float32) for xi in x])
        return self.fc(q_out)  # Map quantum features to class probabilities


# Initialize Model, Loss, Optimizer
model = QuantumClassifier()
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate)


In [ ]:
# import pennylane as qml
# import numpy as np
# import torch
# from torch import nn
# from torch.optim import Adam
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# # Start tracking AWS Braket task costs
# from braket.tracking import Tracker
# t = Tracker().start()

# # Hyperparameters
# num_qubits = 6       # 6 qubits
# num_layers = 1       # Reduced depth for speed
# num_classes = 5      # 5-class classification
# epochs = 1         # Fewer epochs for faster training
# batch_size = 16      # Mini-batch size
# learning_rate = 0.01 # Learning rate

# # Define AWS Braket Quantum Device (SV1 simulator)
# device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
# dev = qml.device('braket.aws.qubit', device_arn=device_arn, wires=num_qubits, shots=100)

# # Quantum Circuit with Optimized Speed
# @qml.qnode(dev, interface="torch")
# def quantum_circuit(inputs, weights):
#     # Faster Encoding: AngleEmbedding (Y rotations)
#     qml.AngleEmbedding(inputs, wires=range(num_qubits), rotation="Y")

#     # Variational Quantum Circuit (VQC)
#     for w in weights:
#         for i in range(num_qubits):
#             qml.RY(w[i], wires=i)
#             qml.RZ(w[i], wires=i)  # Removed RX for speed

#         # Star Entanglement: Faster than All-to-All
#         for i in range(1, num_qubits):
#             qml.CNOT(wires=[0, i])  # Connect all to qubit 0

#     return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]  # Expectation values

# # Define Torch Model with Quantum Layer
# class QuantumClassifier(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.q_weights = nn.Parameter(0.01 * torch.randn(num_layers, num_qubits))  # Trainable weights
#         self.fc = nn.Linear(num_qubits, num_classes)  # Fully connected layer

#     def forward(self, x):
#         # Compute quantum circuit output for each input in batch
#         q_out = torch.stack([torch.tensor(quantum_circuit(xi, self.q_weights), dtype=torch.float32) for xi in x])
#         return self.fc(q_out)  # Map quantum features to class probabilities


# # Initialize Model, Loss, Optimizer
# model = QuantumClassifier()
# loss_fn = nn.CrossEntropyLoss()
# optimizer = Adam(model.parameters(), lr=learning_rate)


In [ ]:
# loss_values = []
# accuracy_values = []
# for epoch in range(epochs):
#     optimizer.zero_grad()
#     y_pred = model(X_train)
#     loss = loss_fn(y_pred, y_train)
#     loss.backward()
#     optimizer.step()

#     correct = (y_pred.argmax(dim=1) == y_train).sum().item()
#     accuracy = correct / len(y_train) * 100

#     loss_values.append(loss.item())
#     accuracy_values.append(accuracy)

#     print(f"Epoch {epoch} | Loss: {loss.item():.4f} | Accuracy: {accuracy:.2f}%")



In [ ]:
loss_values = []
accuracy_values = []

for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    loss.backward()
    optimizer.step()

    correct = (y_pred.argmax(dim=1) == y_train).sum().item()
    accuracy = correct / len(y_train) * 100

    loss_values.append(loss.item())
    accuracy_values.append(accuracy)

    # **Track cost at each epoch**
    print(f"Epoch {epoch} | Loss: {loss.item():.4f} | Accuracy: {accuracy:.2f}%")
    print("Quantum Task Summary:", t.quantum_tasks_statistics())
    print(f"Estimated cost so far: {t.qpu_tasks_cost() + t.simulator_tasks_cost():.3f} USD\n")


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix
import pennylane as qml

In [ ]:
from sklearn.metrics import classification_report

# Evaluate on Test Dataset
model.eval()  # Set the model to evaluation mode

# Make predictions on the test dataset
with torch.no_grad():  # No need to compute gradients for evaluation
    y_pred_test = model(X_test)  # Get predictions for the test data
    predicted_labels_test = y_pred_test.argmax(dim=1)  # Get the predicted class labels

    # Calculate test accuracy
    test_accuracy = (predicted_labels_test == y_test).float().mean().item() * 100

# Print test accuracy in a structured format
print("\n" + "="*70)
print(f"🚀 Final Test Accuracy: {test_accuracy:.2f}%")
print("="*70 + "\n")

# --- CLASSIFICATION REPORT ---
# Convert numerical labels to class names
class_labels = encoder.classes_  # Use actual class names

# Generate classification report
report = classification_report(y_test.numpy(), predicted_labels_test.numpy(),
                               target_names=class_labels, digits=4)

# Print classification report with formatting
print("📌 Classification Report on Test Dataset:")
print("-" * 70)
print(report)
print("-" * 70)


In [ ]:
# Create high-resolution figure
plt.figure(figsize=(9, 7), dpi=300)

# Plot Training Loss
plt.plot(loss_values, label="Training Loss", color="#d62728", linewidth=2.5, linestyle="-")

# Plot Training Accuracy
plt.plot(accuracy_values, label="Training Accuracy", color="#1f77b4", linewidth=2.5, linestyle="--")

# Labels & Title
plt.xlabel("Epochs", fontsize=14, fontweight="bold", labelpad=10)
plt.ylabel("Value", fontsize=14, fontweight="bold", labelpad=10)
plt.title("Training Loss & Accuracy Over Epochs", fontsize=16, fontweight="bold", pad=15)

# Customize legend
plt.legend(fontsize=12, loc="best", frameon=True, shadow=True, fancybox=True)

# Add grid for better readability
plt.grid(True, linestyle="--", alpha=0.6)

# Set tick parameters for clarity
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Show plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

# Binarize the labels
y_test_binarized = label_binarize(y_test.numpy(), classes=np.arange(num_classes))
y_pred_prob = torch.softmax(y_pred_test, dim=1).numpy()

# Define colors for better distinction
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]

# Create a high-resolution figure
plt.figure(figsize=(10, 8), dpi=300)

# Plot ROC curves for each class
for i, color in zip(range(num_classes), colors):
    fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_pred_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=color, lw=2, label=f"{class_labels[i]} (AUC = {roc_auc:.2f})")

# Plot the diagonal reference line
plt.plot([0, 1], [0, 1], "k--", lw=2, label="Random Classifier (AUC = 0.50)")

# Labels & Title
plt.xlabel("False Positive Rate", fontsize=14, fontweight="bold", labelpad=10)
plt.ylabel("True Positive Rate", fontsize=14, fontweight="bold", labelpad=10)
plt.title("Receiver Operating Characteristic (ROC) Curve", fontsize=16, fontweight="bold", pad=15)

# Customize legend
plt.legend(fontsize=12, loc="lower right", frameon=True, shadow=True, fancybox=True)

# Add grid for better readability
plt.grid(True, linestyle="--", alpha=0.6)

# Show the plot
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Compute Confusion Matrix
conf_matrix = confusion_matrix(y_test.numpy(), predicted_labels_test.numpy())

# Define class labels from encoder
class_labels = encoder.classes_

# Create a high-resolution figure
plt.figure(figsize=(10, 8), dpi=300)

# Plot Confusion Matrix with enhanced styling
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="viridis",
            xticklabels=class_labels, yticklabels=class_labels,
            linewidths=0.5, linecolor="gray", cbar=True, annot_kws={"size": 14})

# Add Labels and Title with Improved Font Styling
plt.xlabel("Predicted Label", fontsize=14, fontweight="bold", labelpad=10)
plt.ylabel("True Label", fontsize=14, fontweight="bold", labelpad=10)
plt.title("Confusion Matrix", fontsize=16, fontweight="bold", pad=15)

# Customize tick labels
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12, rotation=0)

# Show the plot
plt.show()


In [ ]:
# --- QUANTUM CIRCUIT DIAGRAM (Graphical) ---
import matplotlib.pyplot as plt

fig, ax = qml.draw_mpl(quantum_circuit)(torch.rand(num_qubits), model.q_weights.detach().numpy())
plt.show()


In [ ]:
# Ensure model is in evaluation mode
model.eval()

# Compute validation loss on test set
with torch.no_grad():
    y_val_pred = model(X_test)
    val_loss_values = [loss_fn(y_val_pred, y_test).item()] * len(loss_values)  # Repeat same loss for all epochs

In [ ]:
import matplotlib.pyplot as plt

# Create the figure with high resolution for publication
plt.figure(figsize=(8, 6), dpi=300)

# Plot Training Loss with smoother line
plt.plot(loss_values, label="Training Loss", color="#E74C3C", linewidth=2.5, marker="o", markersize=4, alpha=0.8)

# Plot Estimated Validation Loss with dashed line
plt.plot(val_loss_values, label="Validation Loss", color="#3498DB", linestyle="--", linewidth=2.5, marker="s", markersize=4, alpha=0.8)

# Set labels with LaTeX-like formatting
plt.xlabel("Epochs", fontsize=14, fontweight="bold")
plt.ylabel("Loss (Cross-Entropy)", fontsize=14, fontweight="bold")

# Set title with larger font
plt.title("Training vs. Validation Loss Over Epochs", fontsize=16, fontweight="bold", pad=15)

# Customize axes ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add grid with better styling
plt.grid(True, linestyle="--", alpha=0.5)

# Add a legend with improved visibility
plt.legend(fontsize=12, loc="upper right", frameon=True, shadow=True, fancybox=True)

# Show the plot
plt.show()


In [ ]:
print(f"Number of shots used: {dev.shots}")

In [ ]:
# --- MODEL WEIGHTS ---
print("\nModel Weights:")
print(model.q_weights.detach().numpy())


In [ ]:
# # Save the trained model weights
# model_path = "/content/quantum_model.pth"
# torch.save(model.state_dict(), model_path)
# print(f"Model weights saved to {model_path}")

In [ ]:
# from google.colab import files
# files.download(model_path)

In [ ]:
# checkpoint = {
#     "epoch": epochs,  # Last epoch number
#     "model_state": model.state_dict(),  # Model weights
#     "optimizer_state": optimizer.state_dict(),  # Optimizer state
# }

# torch.save(checkpoint, "quantum_model_deepSave.pth")  # Save everything
# print("Quantum model saved successfully!")


In [ ]:
# checkpoint = {
#     "epoch": epochs,  # Last epoch number
#     "model_state": model.state_dict(),  # Model weights
#     "optimizer_state": optimizer.state_dict(),  # Optimizer state
#     "loss_values": loss_values,  # Save loss history
#     "accuracy_values": accuracy_values,  # Save accuracy history
# }

# torch.save(checkpoint, "quantum_model_deepSave.pth")  # Save all in one file
# print("✅ Quantum model & training history saved successfully!")


In [ ]:
# import torch

# checkpoint = {
#     "epoch": epochs,  # Last epoch number
#     "model_class": model.__class__.__name__,  # Save model class name
#     "model_state": model.state_dict(),  # Model weights
#     "optimizer_state": optimizer.state_dict(),  # Optimizer state
#     # "scheduler_state": scheduler.state_dict() if scheduler else None,  # LR scheduler (optional)
#     "loss_values": loss_values,  # Loss history
#     "accuracy_values": accuracy_values,  # Accuracy history
#     "gradient_norms": [p.grad.norm().item() for p in model.parameters() if p.grad is not None],  # Gradients (optional)
#     "hyperparameters": {
#         "learning_rate": optimizer.param_groups[0]["lr"],
#         "batch_size": batch_size,
#         "num_epochs": epochs,
#         "architecture": str(model),
#     },  # Store hyperparameters
# }

# torch.save(checkpoint, "quantum_model_full_save.pth")  # Save all in one file
# print("✅ Full model, training history, and hyperparameters saved successfully!")


## usage

In [ ]:
# # # Reinitialize model and optimizer
# # model = QuantumClassifier()
# # optimizer = Adam(model.parameters(), lr=learning_rate)

# # # Load checkpoint
# # checkpoint = torch.load("quantum_model.pth")
# # model.load_state_dict(checkpoint["model_state"])
# # optimizer.load_state_dict(checkpoint["optimizer_state"])
# # start_epoch = checkpoint["epoch"]  # Resume from last epoch

# # model.eval()  # Set to evaluation mode
# # print(f"Model loaded. Resuming from epoch {start_epoch}.")

# import torch
# from tabulate import tabulate

# # Load checkpoint
# checkpoint = torch.load("quantum_model_deepSave.pth")

# # Load model and set to evaluation mode
# model = QuantumClassifier()
# model.load_state_dict(checkpoint["model_state"])
# model.eval()

# # Extract training history
# loss_values = checkpoint["loss_values"]
# accuracy_values = checkpoint["accuracy_values"]
# epochs = checkpoint["epoch"]

# # Prepare data for tabular display (Epoch starts from 1 instead of 0)
# table_data = [[epoch + 1, f"{loss_values[epoch]:.4f}", f"{accuracy_values[epoch]:.2f}%"]
#               for epoch in range(len(loss_values))]

# # Print formatted table
# print("\n📌 Training Progress Overview:\n")
# print(tabulate(table_data, headers=["Epoch", "Loss", "Accuracy"], tablefmt="grid"))

# # Save to file for publication (Optional)
# with open("training_log.txt", "w") as f:
#     f.write(tabulate(table_data, headers=["Epoch", "Loss", "Accuracy"], tablefmt="grid"))

# print("\n✅ Training log saved as 'training_log.txt' for publication!")